In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2\

import networkx as nx
import numpy as np
import tensorflow as tf

from keras import backend as K
from keras.layers import Dense
from keras.models import Model, Sequential


from TCGAMultiOmics.multiomics import MultiOmicsData
from moge.network.heterogeneous_network import HeterogeneousNetwork


Using TensorFlow backend.


#  Import network from file

In [2]:
import pickle

# WRITE
# with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0.pickle', 'wb') as file:
#     pickle.dump(network, file)

# READ
with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0.pickle', 'rb') as file:
# with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_biogrid.pickle', 'rb') as file:
    network = pickle.load(file)
    network.remove_extra_nodes()
#     network.node_list = network.all_nodes
#     node_list = network.node_list

In [3]:
# READ edgelists
with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0_test_edges.pickle', 'rb') as file:
    test_edges_dict = pickle.load(file)
    
with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0_val_edges.pickle', 'rb') as file:
    val_edges_dict = pickle.load(file)

# Load training data

In [4]:
X, y = network.multi_omics_data.load_data(modalities=["MIR", "GE"])

In [5]:
X["MIR"].shape

(460, 1870)

# Training Source Target Graph Embedding

In [49]:
from keras.layers import Input, Conv1D, Lambda, merge, Dense, Flatten, MaxPooling1D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy

def W_init(shape,name=None):
    """Initialize weights as in paper"""
    values = np.random.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)
#//TODO: figure out how to initialize layer biases in keras.
def b_init(shape,name=None):
    """Initialize bias as in paper"""
    values=np.random.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)


In [11]:
tf.reset_default_graph()
# sess.close()
sess = tf.InteractiveSession()

In [35]:
node_features_size = X["MIR"].shape[0]
input_shape = (node_features_size, 1)
_d = 128
n_nodes = len(network.node_list)

In [36]:
merge.Subtract

460

In [54]:
with tf.name_scope('inputs'):
    E_ij = tf.placeholder(tf.float32, shape=(1,), name="E_ij")
#     input_i = tf.placeholder(tf.float32, shape=input_shape, name="input_i")
#     input_j = tf.placeholder(tf.float32, shape=input_shape, name="input_j")
    input_i = Input(input_shape, name="input_i")
    input_j = Input(input_shape, name="input_j")
    is_directed = tf.placeholder(tf.bool, name="is_directed")
    i = tf.Variable(int, name="i", trainable=False)
    j = tf.Variable(int, name="j", trainable=False)

In [55]:
#build convnet to use in each siamese 'leg'
convnet = Sequential()
convnet.add(Conv1D(64, (5), activation='relu',input_shape=input_shape, 
                   kernel_initializer=W_init, kernel_regularizer=l2(2e-4)))

convnet.add(MaxPooling1D())
convnet.add(Conv1D(128, (7), activation='relu', kernel_regularizer=l2(2e-4),
                   kernel_initializer=W_init, bias_initializer=b_init))
convnet.add(MaxPooling1D())
convnet.add(Conv1D(128, (4), activation='relu', kernel_initializer=W_init, 
                   kernel_regularizer=l2(2e-4), bias_initializer=b_init))
convnet.add(MaxPooling1D())
convnet.add(Conv1D(256, (4), activation='relu', kernel_initializer=W_init,
                   kernel_regularizer=l2(2e-4), bias_initializer=b_init))
convnet.add(Flatten())
convnet.add(Dense(4096,activation="sigmoid", kernel_regularizer=l2(1e-3),
                  kernel_initializer=W_init, bias_initializer=b_init))

#encode each of the two inputs into a vector with the convnet
encoded_l = convnet(input_i)
encoded_r = convnet(input_j)

#merge two encoded inputs with the l1 distance between them
L1_distance = lambda x: K.abs(x[0]-x[1])
both = K.abs(merge.subtract([encoded_l, encoded_r]))
print(both)
prediction = Dense(1, activation='sigmoid', bias_initializer=b_init)(both)
siamese_net = Model(input=[input_i, input_j], output=prediction)

Tensor("Abs_2:0", shape=(?, 4096), dtype=float32)


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


AttributeError: 'NoneType' object has no attribute '_inbound_nodes'

In [13]:
optimizer = Adam(0.00006)
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

siamese_net.count_params()


NameError: name 'siamese_net' is not defined

In [126]:
with tf.name_scope('siamese'):
    siamese = Dense(128, activation='relu')(N_i)
    siamese = Dense(_d, activation='relu')(siamese)
    
    emb_c_i = siamese(N_i)
    emb_c_i = siamese(N_j)


In [127]:
with tf.name_scope('embedding'):
    emb_s = tf.Variable(initial_value=tf.random_uniform([n_nodes, int(_d/2)], -1, 1),
                        validate_shape=True, dtype=tf.float32,
                        name="emb_t", trainable=True)

    emb_t = tf.Variable(initial_value=tf.random_uniform([n_nodes, int(_d/2)], -1, 1),
                        validate_shape=True, dtype=tf.float32,
                        name="emb_s", trainable=True)

    emb_c = tf.concat([emb_s, emb_t], axis=1, name="emb_concat")

In [128]:
emb_s = emb_s[i].assign(tf.reshape(siamese[:, 0 : int(_d/2)], [-1]))
emb_t = emb_t[i].assign(tf.reshape(siamese[:, int(_d/2) : _d], [-1]))

# with tf.control_dependencies([emb_s[i].assign(emb_c_i)]):
#     emb_s = tf.identity(emb_s)

In [ ]:
sess.as_default()
K.set_session(sess)
session.run(init_op)